In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Define basic parameters
types_of_panels = ['Monocrystalline', 'Polycrystalline', 'Thin-film']
efficiency = {'Monocrystalline': 0.15, 'Polycrystalline': 0.13, 'Thin-film': 0.07}  # Lower end of efficiency
fixed_cost = {'Monocrystalline': 0.75, 'Polycrystalline': 0.70, 'Thin-film': 0.30}  # Lower end of cost per kW
install_cost = {'Monocrystalline': 300, 'Polycrystalline': 250, 'Thin-film': 200}  # Example installation costs
maintenance_cost_percentage = 0.01  # 1% for large systems
panel_area_ft2 = 15  # Area in ft²
panel_area_m2 = panel_area_ft2 * 0.092903  # Convert to m²

In [3]:
available_space_m2 = 2559.5
demand_kW = 1000  # Example demand

In [4]:
# Create a new model
model = gp.Model("solar_optimization")

# Decision variables
x = model.addVars(types_of_panels, vtype=GRB.INTEGER, name="x")
y = model.addVars(types_of_panels, vtype=GRB.BINARY, name="y")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22


In [5]:
# Objective function: Minimize total cost
model.setObjective(
    gp.quicksum(
        (fixed_cost[j] + install_cost[j] + maintenance_cost_percentage * (fixed_cost[j] + install_cost[j])) * x[j]
        for j in types_of_panels
    ),
    GRB.MINIMIZE
)

# Constraints
# Demand constraint: Total generated power must meet demand
model.addConstr(
    gp.quicksum(efficiency[j] * x[j] * panel_area_m2 for j in types_of_panels) >= demand_kW,
    name="demand"
)

# Space constraint: Total used space must not exceed available space
model.addConstr(
    gp.quicksum(x[j] * panel_area_m2 for j in types_of_panels) <= available_space_m2,
    name="space"
)

# Installation constraint: Panels can only be installed if selected
M = 1000  # A large number
for j in types_of_panels:
    model.addConstr(x[j] <= M * y[j], name=f"install_{j}")

# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5825U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x7792be93
Variable types: 0 continuous, 6 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+03]
  Objective range  [2e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 3e+03]
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [6]:
# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for j in types_of_panels:
        print(f"{j}: {x[j].x} panels, Build: {y[j].x}")
    print(f"Total Cost: ${model.objVal}")
else:
    print("No optimal solution found.")

No optimal solution found.
